In [ ]:
!mkdir -p ~/.kaggle      # setting the directory of kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d salader/dogs-vs-cats  # downloading the dataset from kaggle

100% 1.06G/1.06G [00:07<00:00, 196MB/s]
100% 1.06G/1.06G [00:07<00:00, 154MB/s]


In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
import zipfile          # Unzipping the file download from kaggle
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import tensorflow as tf    # importing the dependencies
from tensorflow import keras
from keras import Sequential
from keras.layers import Conv2D,Dense,MaxPooling2D,Flatten,BatchNormalization,Dropout
from keras.applications.vgg16 import VGG16

In [ ]:
#Making object of VGG16
conv_base=VGG16(
    weights="imagenet",
    include_top=False,
    input_shape=(256,256,3)
)

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
#summary of the model
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [ ]:
# Freezing the model
conv_base.trainable=False

In [ ]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [ ]:
# Adding the ANN part to the end of VGG16
model=Sequential()

model.add(conv_base)
model.add(Flatten())
model.add(Dense(256,activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(1,activation="sigmoid"))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 256)               8388864   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 23,103,809
Trainable params: 8,389,121
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
#Making Train and Test data
train_data=keras.utils.image_dataset_from_directory(
    directory="/content/train",
    labels="inferred",
    label_mode="int",
    batch_size=32,
    image_size=(256,256)
)

test_data=keras.utils.image_dataset_from_directory(
    directory="/content/test",
    labels="inferred",
    label_mode="int",
    batch_size=32,
    image_size=(256,256)
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [ ]:
# Standardizing the image size
def process(image,label):
  image=tf.cast(image/255,tf.float32)

  return image,label

train_data=train_data.map(process)
test_data=test_data.map(process)

In [ ]:
# compiling the model
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [ ]:
# train the model
history=model.fit(train_data,epochs=10,validation_data=(test_data))

Epoch 1/10
625/625 [==============================] - 151s 218ms/step - loss: 0.3251 - accuracy: 0.8876 - val_loss: 0.1742 - val_accuracy: 0.9246
Epoch 2/10
625/625 [==============================] - 143s 228ms/step - loss: 0.1647 - accuracy: 0.9340 - val_loss: 0.1435 - val_accuracy: 0.9394
Epoch 3/10
625/625 [==============================] - 142s 227ms/step - loss: 0.1348 - accuracy: 0.9444 - val_loss: 0.1480 - val_accuracy: 0.9356
Epoch 4/10
625/625 [==============================] - 126s 201ms/step - loss: 0.1223 - accuracy: 0.9503 - val_loss: 0.1449 - val_accuracy: 0.9352
Epoch 5/10
385/625 [=================>............] - ETA: 38s - loss: 0.1200 - accuracy: 0.9500

In [ ]:
# testing the model with some random images
import cv2
from google.colab.patches import cv2_imshow

img = cv2.imread("/content/dog-puppy-on-garden-royalty-free-image-1586966191.jpg")

cv2_imshow(img)

def prediction(img):
  img_sh=cv2.resize(img,(256,256))
  test_img=img_sh.reshape((1,256,256,3))

  output=model.predict(test_img)
  ot=int(output[0][0])

  if ot==1:
    print("Dog")

  else:
    print("Cat")

prediction(img)

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

img = cv2.imread("/content/dog-puppy-on-garden-royalty-free-image-1586966191.jpg")

abhi=pickle.load(open("cat_vs_dog1.pkl","rb"))

cv2_imshow(img)

def prediction(img):
  img_sh=cv2.resize(img,(256,256))
  test_img=img_sh.reshape((1,256,256,3))

  output=abhi.predict(test_img)
  ot=int(output[0][0])

  if ot==1:
    print("Dog")

  else:
    print("Cat")

prediction(img)

In [ ]:
# Save the model
tf.keras.models.save_model(model,"my_model.hdf5")

In [ ]:
# load the model
model=tf.keras.models.load_model("my_model.hdf5")


In [ ]:
# applying quantization process to the model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Apply default optimizations
quantized_model = converter.convert()

In [ ]:
with open('quantized_model.tflite', 'wb') as f:
    f.write(quantized_model)

In [ ]:
# Load the quantized model
interpreter = tf.lite.Interpreter(model_path='quantized_model.tflite')
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
# again testing the quantized model
given_image="international-cat-day1-scaled.jpg"
image=Image.open(given_image)

In [ ]:
size=(256,256)
image=ImageOps.fit(image,size,Image.ANTIALIAS)
image=np.asarray(image)
img_reshape=image[np.newaxis,...]

In [ ]:

image = np.array(img_reshape, dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], image)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
ot=int(output_data[0][0])
print(ot)
if ot==1:
    print("Dog")

else:
    print("Cat")